In [36]:
import pandas as pd
import matplotlib.pyplot as plt

# Step 1: Load data with MultiIndex
df = pd.read_csv('crypto.csv', header=[0, 1], index_col=0, parse_dates=[0])

# Step 2: Rename all columns using the join method
df.columns = ['_'.join(col).strip() for col in df.columns]

# Step 3: Select only 'close' and 'volume' columns
coins = ['ada', 'btc', 'doge', 'eth', 'sol', 'xrp']
close_columns = [f'close_{coin}' for coin in coins]
volume_columns = [f'volume_{coin}' for coin in coins]
df_subset = df[close_columns + volume_columns].copy()

# Step 4: Handle missing values (interpolate for simplicity)
df_subset = df_subset.interpolate(method='linear')

# Step 5: Calculate daily returns for each coin using a loop
for coin in coins:
    df_subset[f'returns_{coin}'] = df_subset[f'close_{coin}'].pct_change() * 100

# Step 6: Calculate mu and sigma for each coin
mu_sigma = {}
for coin in coins:
    mu_sigma[coin] = {
        'mu': df_subset[f'returns_{coin}'].mean(),
        'sigma': df_subset[f'returns_{coin}'].std()
    }

# Step 7: Calculate volume weights
volume_means = df_subset[[f'volume_{coin}' for coin in coins]].mean()
normalized_volumes = volume_means / volume_means.max()

# Step 8: Investment plan (allocate 10,000 RUB) with volume
total_budget = 10000
allocation = {}
for coin in mu_sigma:
    weight = (mu_sigma[coin]['mu'] / mu_sigma[coin]['sigma'] * normalized_volumes[f'volume_{coin}']) if mu_sigma[coin]['sigma'] != 0 else 0
    allocation[coin] = min(5000, max(0, total_budget * weight / sum((m['mu'] / m['sigma'] * normalized_volumes[f'volume_{c}']) if m['sigma'] != 0 else 0 for c, m in mu_sigma.items())))
total_allocated = sum(allocation.values())
for coin in allocation:
    allocation[coin] = (allocation[coin] / total_allocated) * 10000

print("\nInvestment Allocation (RUB):")
for coin, amount in allocation.items():
    print(f"{coin.upper()}: {amount:.2f}")

# Step 9: Visualize returns
plt.figure(figsize=(12, 6))
for coin in coins:
    plt.plot(df_subset.index, df_subset[f'returns_{coin}'], label=f'{coin.upper()} Returns')
plt.title('Daily Returns of Major Coins')
plt.xlabel('Date')
plt.ylabel('Returns (%)')
plt.legend()
plt.grid(True)
plt.savefig('major_coins_returns.png')
plt.close()


Investment Allocation (RUB):
ADA: 247.58
BTC: 5416.53
DOGE: 286.41
ETH: 3075.76
SOL: 431.51
XRP: 542.21


In [37]:
import pandas as pd

In [38]:
df = pd.read_csv('crypto.csv', header=[0, 1], index_col=0, parse_dates=[0])
df.head()

Price          close                                                   \
Ticker           ada          btc      doge         eth sol       xrp   
Date                                                                    
2020-01-01  0.033458  7200.174316  0.002033  130.802002 NaN  0.192667   
2020-01-02  0.032751  6985.470215  0.002009  127.410179 NaN  0.188043   
2020-01-03  0.034180  7344.884277  0.002145  134.171707 NaN  0.193521   
2020-01-04  0.034595  7410.656738  0.002241  135.069366 NaN  0.194355   
2020-01-05  0.034721  7411.317383  0.002419  136.276779 NaN  0.195537   

Price           High                                     ...      Open  \
Ticker           ada          btc      doge         eth  ...      doge   
Date                                                     ...             
2020-01-01  0.033813  7254.330566  0.002052  132.835358  ...  0.002028   
2020-01-02  0.033507  7212.155273  0.002110  130.820038  ...  0.002034   
2020-01-03  0.034427  7413.715332  0.002177  134.554016  ...  0.002008   
2020-01-04  0.034685  7427.385742  0.002490  136.052719  ...  0.002144   
2020-01-05  0.035356  7544.497070  0.002491  139.410202  ...  0.002240   

Price                                   volume                         \
Ticker             eth sol       xrp       ada          btc      doge   
Date                                                                    
2020-01-01  129.630661 NaN  0.192912  22948374  18565664997  51180941   
2020-01-02  130.820038 NaN  0.192708  20843934  20802083465  65071106   
2020-01-03  127.411263 NaN  0.187948  30162644  28111481032  62619988   
2020-01-04  134.168518 NaN  0.193521  29535781  18444271275  94227582   
2020-01-05  135.072098 NaN  0.194367  21479178  19725074095  52631740   

Price                                    
Ticker              eth sol         xrp  
Date                                     
2020-01-01   7935230330 NaN  1041134003  
2020-01-02   8032709256 NaN  1085351426  
2020-01-03  10476845358 NaN  1270017043  
2020-01-04   7430904515 NaN   999331594  
2020-01-05   7526675353 NaN  1168067557  

[5 rows x 30 columns]

In [39]:
df.columns = ['_'.join(col).strip() for col in df.columns]
df.head()

,close_ada,close_btc,close_doge,close_eth,close_sol,close_xrp,High_ada,High_btc,High_doge,High_eth,...,Open_doge,Open_eth,Open_sol,Open_xrp,volume_ada,volume_btc,volume_doge,volume_eth,volume_sol,volume_xrp
Date,,,,,,,,,,,,,,,,,,,,,
2020-01-01,0.033458,7200.174316,0.002033,130.802002,NaN,0.192667,0.033813,7254.330566,0.002052,132.835358,...,0.002028,129.630661,NaN,0.192912,22948374,18565664997,51180941,7935230330,NaN,1041134003
2020-01-02,0.032751,6985.470215,0.002009,127.410179,NaN,0.188043,0.033507,7212.155273,0.002110,130.820038,...,0.002034,130.820038,NaN,0.192708,20843934,20802083465,65071106,8032709256,NaN,1085351426
2020-01-03,0.034180,7344.884277,0.002145,134.171707,NaN,0.193521,0.034427,7413.715332,0.002177,134.554016,...,0.002008,127.411263,NaN,0.187948,30162644,28111481032,62619988,10476845358,NaN,1270017043
2020-01-04,0.034595,7410.656738,0.002241,135.069366,NaN,0.194355,0.034685,7427.385742,0.002490,136.052719,...,0.002144,134.168518,NaN,0.193521,29535781,18444271275,94227582,7430904515,NaN,999331594
2020-01-05,0.034721,7411.317383,0.002419,136.276779,NaN,0.195537,0.035356,7544.497070,0.002491,139.410202,...,0.002240,135.072098,NaN,0.194367,21479178,19725074095,52631740,7526675353,NaN,1168067557


In [40]:
coins = ['ada', 'btc', 'doge', 'eth', 'sol', 'xrp']
close_columns = [f'close_{coin}' for coin in coins]
close_columns

['close_ada', 'close_btc', 'close_doge', 'close_eth', 'close_sol', 'close_xrp']

In [41]:
volume_columns = [f'volume_{coin}' for coin in coins]
volume_columns

['volume_ada',
 'volume_btc',
 'volume_doge',
 'volume_eth',
 'volume_sol',
 'volume_xrp']

In [42]:
df_subset = df[close_columns + volume_columns].copy()
df_subset.head()

,close_ada,close_btc,close_doge,close_eth,close_sol,close_xrp,volume_ada,volume_btc,volume_doge,volume_eth,volume_sol,volume_xrp
Date,,,,,,,,,,,,
2020-01-01,0.033458,7200.174316,0.002033,130.802002,NaN,0.192667,22948374,18565664997,51180941,7935230330,NaN,1041134003
2020-01-02,0.032751,6985.470215,0.002009,127.410179,NaN,0.188043,20843934,20802083465,65071106,8032709256,NaN,1085351426
2020-01-03,0.034180,7344.884277,0.002145,134.171707,NaN,0.193521,30162644,28111481032,62619988,10476845358,NaN,1270017043
2020-01-04,0.034595,7410.656738,0.002241,135.069366,NaN,0.194355,29535781,18444271275,94227582,7430904515,NaN,999331594
2020-01-05,0.034721,7411.317383,0.002419,136.276779,NaN,0.195537,21479178,19725074095,52631740,7526675353,NaN,1168067557


In [43]:
for coin in coins:
    df_subset[f'return_{coin}'] = df_subset[f'close_{coin}'].pct_change() * 100

df_subset.head()

,close_ada,close_btc,close_doge,close_eth,close_sol,close_xrp,volume_ada,volume_btc,volume_doge,volume_eth,volume_sol,volume_xrp,return_ada,return_btc,return_doge,return_eth,return_sol,return_xrp
Date,,,,,,,,,,,,,,,,,,
2020-01-01,0.033458,7200.174316,0.002033,130.802002,NaN,0.192667,22948374,18565664997,51180941,7935230330,NaN,1041134003,NaN,NaN,NaN,NaN,NaN,NaN
2020-01-02,0.032751,6985.470215,0.002009,127.410179,NaN,0.188043,20843934,20802083465,65071106,8032709256,NaN,1085351426,-2.113087,-2.981929,-1.180530,-2.593097,NaN,-2.400000
2020-01-03,0.034180,7344.884277,0.002145,134.171707,NaN,0.193521,30162644,28111481032,62619988,10476845358,NaN,1270017043,4.363222,5.145166,6.769543,5.306898,NaN,2.913161
2020-01-04,0.034595,7410.656738,0.002241,135.069366,NaN,0.194355,29535781,18444271275,94227582,7430904515,NaN,999331594,1.214163,0.895487,4.475526,0.669038,NaN,0.430963
2020-01-05,0.034721,7411.317383,0.002419,136.276779,NaN,0.195537,21479178,19725074095,52631740,7526675353,NaN,1168067557,0.364205,0.008915,7.942881,0.893920,NaN,0.608168


In [44]:
# Step 4: Trim data to start from the first non-NaN value for all coins

first_valid_date = df_subset[close_columns].apply(lambda x: x.first_valid_index()).max()
df_subset = df_subset[df_subset.index >= first_valid_date]
df_subset.head()

,close_ada,close_btc,close_doge,close_eth,close_sol,close_xrp,volume_ada,volume_btc,volume_doge,volume_eth,volume_sol,volume_xrp,return_ada,return_btc,return_doge,return_eth,return_sol,return_xrp
Date,,,,,,,,,,,,,,,,,,
2020-04-10,0.033215,6865.493164,0.001968,158.412445,0.951054,0.187464,95977717,43622840992,122108790,17980944616,87364276.0,2105282079,-8.240787,-5.979058,-2.622461,-7.256546,NaN,-5.443944
2020-04-11,0.033354,6859.083008,0.001990,158.216019,0.776819,0.188063,75648111,31222085946,161367396,13555089447,43862444.0,1847482831,0.418480,-0.093368,1.117883,-0.123997,-18.320200,0.319526
2020-04-12,0.033806,6971.091797,0.001990,161.142426,0.882507,0.190135,73751090,35759567632,169892709,15123721386,38736897.0,2087857417,1.355162,1.632999,0.000000,1.849627,13.605233,1.101762
2020-04-13,0.033107,6845.037598,0.001959,156.279556,0.777832,0.187732,84000834,38619308647,167357650,16465282133,18211285.0,2253497429,-2.067676,-1.808242,-1.557789,-3.017746,-11.861102,-1.263842
2020-04-14,0.032974,6842.427734,0.001964,157.596390,0.661925,0.185335,74633336,34110434052,188937615,14723156630,16747614.0,1889687587,-0.401729,-0.038128,0.255234,0.842614,-14.901284,-1.276821


In [45]:
mu_sigma = {}

for coin in coins:
    mu_sigma[coin] = {
        'mu': df_subset[f'return_{coin}'].mean(),
        'sigma': df_subset[f'return_{coin}'].std()
    }

mu_sigma

{'ada': {'mu': np.float64(0.29804756336089583),
  'sigma': np.float64(5.37919379097717)},
 'btc': {'mu': np.float64(0.18381012887078912),
  'sigma': np.float64(3.1740461671346925)},
 'doge': {'mu': np.float64(0.5732987316502824),
  'sigma': np.float64(10.874272116768733)},
 'eth': {'mu': np.float64(0.21605314923838775),
  'sigma': np.float64(4.137930642202885)},
 'sol': {'mu': np.float64(0.49259710170101423),
  'sigma': np.float64(6.728279458071118)},
 'xrp': {'mu': np.float64(0.2851520452198862),
  'sigma': np.float64(5.751958914377488)}}